# Clasificación de Textos

#### Descripción del Data Set

El conjunto de datos de 20 grupos de noticias es una colección de aproximadamente 20,000 documentos de grupos de noticias, divididos (casi) uniformemente entre 20 grupos de noticias diferentes. Según nuestro leal saber y entender, fue originalmente recopilado por Ken Lang, probablemente para su artículo "Newsweeder: Learning to filter netnews", aunque no menciona explícitamente esta colección. La colección de 20 grupos de noticias se ha convertido en un conjunto de datos popular para experimentos en aplicaciones de texto de técnicas de aprendizaje automático, como la clasificación de texto y la agrupación de texto.

Para obtener tiempos de ejecución más rápidos para este primer ejemplo, trabajaremos en un conjunto de datos parcial con solo 4 categorías de las 20 disponibles en el conjunto de datos:

In [1]:
categories = ['alt.atheism', 'soc.religion.christian',
               'comp.graphics', 'sci.med']

Ahora podemos cargar la lista de archivos que coinciden con esas categorías de la siguiente manera:

In [2]:
from sklearn.datasets import fetch_20newsgroups
twenty_train = fetch_20newsgroups(subset='train',categories=categories, shuffle=True, random_state=42)

El conjunto de datos devuelto es un "grupo" scikit-learn: un objeto simple de titular con campos a los que se puede acceder como claves pitón dict o atributos de objeto para mayor comodidad, por ejemplo, target_names contiene la lista de los nombres de categoría solicitados:

In [3]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

Los archivos se cargan en la memoria en el atributo de datos. Como referencia, los nombres de archivo también están disponibles:

In [4]:
len(twenty_train.data)

2257

In [5]:
len(twenty_train.filenames)

2257

Imprimamos las primeras líneas del primer archivo cargado:

In [7]:
print("\n".join(twenty_train.data[0].split("\n")[:3]))



From: sd345@city.ac.uk (Michael Collier)
Subject: Converting images to HP LaserJet III?
Nntp-Posting-Host: hampton


In [8]:
print(twenty_train.target_names[twenty_train.target[0]])

comp.graphics


Los algoritmos de aprendizaje supervisados requerirán una etiqueta de categoría para cada documento en el conjunto de entrenamiento. En este caso, la categoría es el nombre del grupo de noticias que también es el nombre de la carpeta que contiene los documentos individuales.

Para razones de eficiencia de velocidad y espacio, scikit-learn carga el atributo de destino como una matriz de enteros que corresponde al índice del nombre de categoría en la lista de nombres de destino. El ID de entero de categoría de cada muestra se almacena en el atributo de destino:

In [9]:
twenty_train.target[:10]

array([1, 1, 3, 3, 3, 3, 3, 2, 2, 2], dtype=int64)

Es posible recuperar los nombres de categoría de la siguiente manera:

In [10]:
for t in twenty_train.target[:10]:
...     print(twenty_train.target_names[t])

comp.graphics
comp.graphics
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
soc.religion.christian
sci.med
sci.med
sci.med


Puede observar que las muestras se han barajado aleatoriamente (con una semilla RNG fija): esto es útil si selecciona solo las primeras muestras para entrenar rápidamente un modelo y obtener una primera idea de los resultados antes de volver a entrenar en el conjunto de datos completo más adelante .

#### Extracción de Variables de Archivos de Texto

Para realizar el aprendizaje automático en documentos de texto, primero debemos convertir el contenido de texto en vectores de características numéricas.

La forma más intuitiva de hacerlo es la representación de bolsas de palabras:

1. Asignar un número entero fijo a cada palabra que aparece en cualquier documento del conjunto de entrenamiento (por ejemplo, construyendo un diccionario de palabras a índices enteros).
2. para cada documento #i, cuente el número de ocurrencias de cada palabra wy guárdelo en X [i, j] como el valor de la característica #j donde j es el índice de la palabra w en el diccionario.

La representación de bolsas de palabras implica que n_features es el número de palabras distintas en el corpus: este número es típicamente mayor que 100,000.

Si n_samples == 10000, almacenar X como una matriz numpy de tipo float32 requeriría 10000 x 100000 x 4 bytes = 4GB en RAM, que es apenas manejable en las computadoras de hoy.

Afortunadamente, la mayoría de los valores en X serán ceros ya que para un documento dado se usarán menos de un par de miles de palabras distintas. Por esta razón, decimos que las bolsas de palabras suelen ser conjuntos de datos dispersos de gran dimensión. Podemos guardar mucha memoria almacenando solo las partes distintas de cero de los vectores de características en la memoria.

Las matrices scipy.sparse son estructuras de datos que hacen exactamente esto, y scikit-learn tiene soporte incorporado para estas estructuras.

### Tokenizing

El preprocesamiento de texto, la tokenización y el filtrado de stopwords se incluyen en un componente de alto nivel que puede construir un diccionario de características y transformar documentos a vectores de características:

In [12]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape


(2257, 35788)

CountVectorizer admite recuentos de N-gramos de palabras o caracteres consecutivos. Una vez instalado, el vectorizador ha creado un diccionario de índices de características:

In [13]:
count_vect.vocabulary_.get(u'algorithm')

4690

El valor de índice de una palabra en el vocabulario está vinculado a su frecuencia en todo el corpus de entrenamiento.

** De ocurrencias a frecuencias **

El recuento de ocurrencias es un buen comienzo, pero hay un problema: los documentos más largos tendrán valores de recuento promedio más altos que los documentos más cortos, aunque puedan hablar sobre los mismos temas.

Para evitar estas posibles discrepancias, basta con dividir el número de apariciones de cada palabra en un documento por el número total de palabras en el documento: estas nuevas funciones se denominan tf para las Frecuencias a Término.

Otro refinamiento en la parte superior de tf es reducir el peso de las palabras que aparecen en muchos documentos en el corpus y, por lo tanto, son menos informativas que las que ocurren solo en una porción más pequeña del corpus.

Esta reducción de escala se denomina tf-idf para "Frecuencia de término multiplicada por frecuencia inversa de documento".

Tanto tf como tf-idf se pueden calcular de la siguiente manera:

In [15]:
from sklearn.feature_extraction.text import TfidfTransformer
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform(X_train_counts)
X_train_tf.shape


(2257, 35788)

En el código de ejemplo anterior, primero usamos el método fit (..) para ajustar nuestro estimador a los datos y, en segundo lugar, el método transform (..) para transformar nuestra matriz de recuento en una representación tf-idf. Estos dos pasos se pueden combinar para lograr el mismo resultado final más rápido omitiendo el procesamiento redundante. Esto se hace mediante el uso del método fit_transform (..) como se muestra a continuación, y como se menciona en la nota en la sección anterior:

In [16]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2257, 35788)

## Entrenando un Clasifcador

Ahora que tenemos nuestras funciones, podemos entrenar a un clasificador para tratar de predecir la categoría de una publicación. Comencemos con un clasificador Bayes ingenuo, que proporciona una buena línea de base para esta tarea. scikit-learn incluye varias variantes de este clasificador; el más adecuado para el recuento de palabras es la variante multinomial:

In [17]:
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB().fit(X_train_tfidf, twenty_train.target)

Para tratar de predecir el resultado en un documento nuevo, necesitamos extraer las características usando casi la misma cadena de extracción de características que antes. La diferencia es que llamamos transformación en lugar de fit_transform en los transformadores, ya que ya se han ajustado al conjunto de entrenamiento:

In [19]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)

predicted = clf.predict(X_new_tfidf)

for doc, category in zip(docs_new, predicted):
        print('%r => %s' % (doc, twenty_train.target_names[category]))



'God is love' => soc.religion.christian
'OpenGL on the GPU is fast' => comp.graphics


## Contruyendo una Línea Base

Para hacer más fácil trabajar con el clasificador vectorizer => transformer =>, scikit-learn proporciona una clase Pipeline que se comporta como un clasificador compuesto:

In [21]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                    ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),])

Los nombres vector, tf idf y clf (clasificador) son arbitrarios. Veremos su uso en la sección de búsqueda de grillas, a continuación. Ahora podemos entrenar el modelo con un solo comando:

In [23]:
text_clf.fit(twenty_train.data, twenty_train.target)  


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...inear_tf=False, use_idf=True)), ('clf', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

## Evaluando el Rendimiento en el DataSet de Validación

Evaluar la precisión predictiva del modelo es igualmente fácil:

In [24]:
import numpy as np
twenty_test = fetch_20newsgroups(subset='test',
     categories=categories, shuffle=True, random_state=42)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)            


0.83488681757656458

Es decir, logramos un 83.4% de precisión. Veamos si podemos mejorar con una máquina de vectores de soporte lineal (SVM), que es ampliamente considerada como uno de los mejores algoritmos de clasificación de texto (aunque también es un poco más lenta que la ingenua Bayes). Podemos cambiar al alumno simplemente conectando un objeto clasificador diferente a nuestro pipeline:

In [25]:
from sklearn.linear_model import SGDClassifier
>>> text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                      ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                            alpha=1e-3, random_state=42,
                                            max_iter=5, tol=None)),])

text_clf.fit(twenty_train.data, twenty_train.target)  
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)            


0.9127829560585885

In [26]:
from sklearn import metrics
print(metrics.classification_report(twenty_test.target, predicted,target_names=twenty_test.target_names))


                        precision    recall  f1-score   support

           alt.atheism       0.95      0.81      0.87       319
         comp.graphics       0.88      0.97      0.92       389
               sci.med       0.94      0.90      0.92       396
soc.religion.christian       0.90      0.95      0.93       398

           avg / total       0.92      0.91      0.91      1502



In [27]:
metrics.confusion_matrix(twenty_test.target, predicted)


array([[258,  11,  15,  35],
       [  4, 379,   3,   3],
       [  5,  33, 355,   3],
       [  5,  10,   4, 379]], dtype=int64)